In [45]:
!pip install mujoco
!pip install mujoco-mjx
!pip install mediapy

In [46]:
import numpy as np

import jax
import mujoco
# from mujoco import mjx
import mediapy as media
from IPython.display import display, Video

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [47]:
# Load model from file
model = mujoco.MjModel.from_xml_path("./models/quadruped/scene.xml")
data = mujoco.MjData(model)

duration = 4  # (seconds)
framerate = 60  # (Hz)
width, height = 720, 480  # Desired resolution

# Apply random inputs to the actuators
mujoco.mj_resetData(model, data)
data.ctrl[:] = np.random.uniform(-1, 1, data.ctrl.shape)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

# Simulate and display video.
frames = []
with mujoco.Renderer(model, width=width, height=height) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

# Save the video to a file in WebM format using VP8 codec
video_path = "simulation_random.webm"
media.write_video(video_path, frames, fps=framerate, codec='vp8', bps=1e6)

# Display the video in the notebook
display(Video(video_path, embed=True))